In [58]:
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
import cv2
import numpy as np
import math
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, BatchNormalization, MaxPooling2D, Dropout, Flatten, Reshape, LSTM, Dense,GlobalAveragePooling2D
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint, ReduceLROnPlateau
from sklearn.model_selection import train_test_split
import albumentations as A
import torch
import torch.optim as optim
from torch.utils.data import Dataset
from torch.utils.data import random_split, DataLoader
import torch.nn as nn
from torch.optim.lr_scheduler import ReduceLROnPlateau

In [59]:
image_dir = "../input/exam-cheating-detection/4000Final/train/images"
label_dir = "../input/exam-cheating-detection/4000Final/train/labels"

dimension_1 = []
dimension_2 = []
base_path = '../input/exam-cheating-detection/4000Final/train/images'


# for image_name in os.listdir(base_path):
#     img_path = os.path.join(base_path, image_name)
    
   
#     img = cv2.imread(img_path)
    
#     if img is not None:
#         # Get the dimensions of the image
#         d1, d2, d3 = img.shape
#         dimension_1.append(d1)
#         dimension_2.append(d2)
#     else:
#         print(f"Error loading image: {img_path}")

# d1 = np.mean(dimension_1)
# d2 = np.mean(dimension_2)
# print(d1,d2)

# cell = cv2.imread(os.path.join(base_path,'10_JPG_jpg.rf.b44bb58adb9dbb4881687aa7b9e17006.jpg'))
# cell.max()
img_shape = (320,320)

In [60]:
Transform = A.Compose([
        A.HorizontalFlip(p=0.5),
        A.RandomRotate90(p=0.5),
        A.Transpose(p=0.2),
        A.Normalize(normalization="min_max", p=1.0),
        A.RandomBrightnessContrast(p=0.3),
        A.HueSaturationValue(p=0.3),
        A.RGBShift(p=0.2),
        A.CLAHE(p=0.1),
        A.GaussNoise(std_range=(0.1, 0.2), p=0.3), 
        A.ISONoise(p=0.2),
        A.MotionBlur(blur_limit=5, p=0.2),
        A.MedianBlur(blur_limit=3, p=0.1),
        A.GaussianBlur(blur_limit=(3, 5), p=0.2),

        A.RandomScale(scale_limit=0.1, p=0.3),
        A.ShiftScaleRotate(shift_limit=0.05, scale_limit=0.1, rotate_limit=10, border_mode=0, p=0.4),
        
        # Adaptive padding and cropping
        A.PadIfNeeded(min_height=320, min_width=320, border_mode=0),
        A.RandomCrop(height=320, width=320, p=0.8),
        A.Resize(height=320, width=320, p=1.0)
    ], bbox_params=A.BboxParams(format='pascal_voc', label_fields=['class_labels']))

In [61]:
class YOLODataset(Dataset):
    def __init__(self, images_dir, labels_dir, transform=None):
        self.images_dir = images_dir
        self.labels_dir = labels_dir
        self.transform = transform
        self.image_files = [f for f in os.listdir(images_dir) if f.endswith(('.jpg', '.png'))]

    def __len__(self):
        return len(self.image_files)

    def __getitem__(self, idx):
        # Load image
        img_filename = self.image_files[idx]
        img_path = os.path.join(self.images_dir, img_filename)
        image = cv2.imread(img_path)
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        height, width, _ = image.shape

        # Load labels
        label_path = os.path.join(self.labels_dir, os.path.splitext(img_filename)[0] + ".txt")
        bboxes, class_labels = [], []

        if os.path.exists(label_path):
            with open(label_path, 'r') as f:
                for line in f:
                    class_id, x_c, y_c, w, h = map(float, line.strip().split())
                    x_min = max((x_c - w/2) * width, 0)
                    y_min = max((y_c - h/2) * height, 0)
                    x_max = min((x_c + w/2) * width, width - 1)
                    y_max = min((y_c + h/2) * height, height - 1)
                    bboxes.append([x_min, y_min, x_max, y_max])
                    class_labels.append(int(class_id))

        # Apply transforms
        if self.transform:
            transformed = self.transform(image=image, bboxes=bboxes, class_labels=class_labels)
            image = transformed['image']
            bboxes = transformed['bboxes']
            class_labels = transformed['class_labels']

        # Normalize and convert to tensor
        image = image / 255.0
        image = np.transpose(image, (2, 0, 1))  # HWC → CHW
        image = torch.tensor(image, dtype=torch.float32)

        # Targets
        targets = []
        for box, label in zip(bboxes, class_labels):
            targets.append([label] + list(box))
        targets = torch.tensor(targets, dtype=torch.float32)

        return image, targets

In [62]:

dataset = YOLODataset(
    images_dir=image_dir,
    labels_dir=label_dir,
    transform=Transform
)

train_size = int(0.8 * len(dataset))
val_size = len(dataset) - train_size

def collate_fn(batch):
    images, targets = zip(*batch)
    images = torch.stack(images)
    return images, list(targets)

train_dataset, val_dataset = random_split(dataset, [train_size, val_size])
dataloader = DataLoader(dataset, batch_size=4, shuffle=True, collate_fn=collate_fn)
train_loader = DataLoader(
    train_dataset,
    batch_size=4,
    shuffle=True,
    collate_fn=collate_fn
)

val_loader = DataLoader(
    val_dataset,
    batch_size=4,
    shuffle=False,
    collate_fn=collate_fn
)

In [63]:
class DetectionLoss(nn.Module):
    def __init__(self):
        super(DetectionLoss, self).__init__()
        self.class_loss_fn = nn.BCEWithLogitsLoss()
        self.bbox_loss_fn = nn.SmoothL1Loss()

    def forward(self, predictions, targets):
        """
        predictions: [B, 1, 5]
        targets: list of [N_i, 5] tensors, N_i can be different for each image
        We will just use the first target box for each image for now.
        """
        if predictions.dim() == 3 and predictions.shape[1] == 1:
            predictions = predictions.squeeze(1)  # [B, 5]

        # Build tensors for labels and boxes by picking the first box for each image
        batch_size = predictions.shape[0]
        true_labels = []
        true_boxes = []
        for t in targets:
            if t.shape[0] > 0:
                true_labels.append(t[0, 0])
                true_boxes.append(t[0, 1:5])
            else:
                # If no targets, treat as background
                true_labels.append(torch.tensor(0.0, device=predictions.device))
                true_boxes.append(torch.zeros(4, device=predictions.device))
        true_labels = torch.stack(true_labels)  # [B]
        true_boxes = torch.stack(true_boxes)    # [B, 4]

        pred_scores = predictions[:, 0]
        pred_boxes = predictions[:, 1:5]

        class_loss = self.class_loss_fn(pred_scores, true_labels)
        bbox_loss = self.bbox_loss_fn(pred_boxes, true_boxes)

        return class_loss + bbox_loss

class EarlyStopping:
    def __init__(self, patience=5, restore_best_weights=True):
        self.patience = patience
        self.counter = 0
        self.best_loss = float('inf')
        self.best_model_state = None
        self.restore_best_weights = restore_best_weights
        self.early_stop = False

    def __call__(self, val_loss, model):
        if val_loss < self.best_loss:
            self.best_loss = val_loss
            self.counter = 0
            if self.restore_best_weights:
                self.best_model_state = model.state_dict()
        else:
            self.counter += 1
            if self.counter >= self.patience:
                self.early_stop = True
                if self.restore_best_weights:
                    model.load_state_dict(self.best_model_state)

def save_checkpoint(model, epoch, val_metric, best_metric, path='best_model.pth'):
    if val_metric > best_metric:
        print(f"Validation metric improved ({best_metric:.4f} → {val_metric:.4f}). Saving model.")
        torch.save(model.state_dict(), path)
        return val_metric
    return best_metric


class ObjectDetectionCNN(nn.Module):
    def __init__(self):
        super(ObjectDetectionCNN, self).__init__()
        self.features = nn.Sequential(
            nn.Conv2d(3, 32, kernel_size=3, padding=1),
            nn.ReLU(),
            nn.BatchNorm2d(32),
            nn.Conv2d(32, 32, kernel_size=3, padding=1),
            nn.ReLU(),
            nn.BatchNorm2d(32),
            nn.MaxPool2d(2),
            nn.Dropout(0.2),

            nn.Conv2d(32, 64, kernel_size=3, padding=1),
            nn.ReLU(),
            nn.BatchNorm2d(64),
            nn.Conv2d(64, 64, kernel_size=3, padding=1),
            nn.ReLU(),
            nn.BatchNorm2d(64),
            nn.MaxPool2d(2),
            nn.Dropout(0.3),

            nn.Conv2d(64, 128, kernel_size=3, padding=1),
            nn.ReLU(),
            nn.BatchNorm2d(128),
            nn.Conv2d(128, 128, kernel_size=3, padding=1),
            nn.ReLU(),
            nn.BatchNorm2d(128),
            nn.MaxPool2d(2),
            nn.Dropout(0.3),

            nn.Conv2d(128, 256, kernel_size=3, padding=1),
            nn.ReLU(),
            nn.BatchNorm2d(256),
            nn.Conv2d(256, 256, kernel_size=3, padding=1),
            nn.ReLU(),
            nn.BatchNorm2d(256),
            nn.MaxPool2d(2),
            nn.Dropout(0.4),
        )

        self.classifier = nn.Sequential(
            nn.AdaptiveAvgPool2d((1, 1)),
            nn.Flatten(),
            nn.Linear(256, 512),
            nn.ReLU(),
            nn.BatchNorm1d(512),
            nn.Dropout(0.5),
            nn.Linear(512, 5)  # [objectness_score, x_min, y_min, x_max, y_max]
        )

    def forward(self, x):
        x = self.features(x)
        x = self.classifier(x)
        return x.unsqueeze(1)  # [B, 1, 5]




In [64]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = ObjectDetectionCNN().to(device)
num_epochs = 20
optimizer = optim.Adam(model.parameters(), lr=1e-4)
criterion = DetectionLoss()
early_stopping = EarlyStopping(patience=5, restore_best_weights=True)
best_val_loss = float('inf')
scheduler = ReduceLROnPlateau(optimizer, mode='min', factor=0.5, patience=2, min_lr=1e-6, verbose=True)

for epoch in range(num_epochs):
    model.train()
    train_loss = 0.0

    for batch_images, batch_targets in dataloader:
        batch_images = batch_images.to(device)
        batch_targets = [t.to(device) for t in batch_targets]  # already correct in your code!
        outputs = model(batch_images)
        loss = criterion(outputs, batch_targets)

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        train_loss += loss.item()

    model.eval()
    val_loss = 0.0

    with torch.no_grad():
        for val_images, val_targets in val_loader:
            val_images = val_images.to(device)  # Move val images to device INSIDE loop
            val_targets = [t.to(device) for t in val_targets]

            outputs = model(val_images)
            loss = criterion(outputs, val_targets)
            val_loss += loss.item()

    avg_train_loss = train_loss / len(dataloader)
    avg_val_loss = val_loss / len(val_loader)

    print(f"Epoch {epoch+1}: Train Loss = {avg_train_loss:.4f}, Val Loss = {avg_val_loss:.4f}")

    # Early stopping
    early_stopping(avg_val_loss, model)
    if early_stopping.early_stop:
        print("Early stopping triggered.")
        break

    # Reduce LR on plateau
    scheduler.step(avg_val_loss)

Epoch 1: Train Loss = 159.8983, Val Loss = 156.7916
Epoch 2: Train Loss = 157.6777, Val Loss = 156.2656
Epoch 3: Train Loss = 153.0074, Val Loss = 149.4646
Epoch 4: Train Loss = 142.1917, Val Loss = 130.6619
Epoch 5: Train Loss = 125.8504, Val Loss = 114.9515
Epoch 6: Train Loss = 104.3935, Val Loss = 92.5925
Epoch 7: Train Loss = 82.3418, Val Loss = 55.6600
Epoch 8: Train Loss = 63.9871, Val Loss = 52.7641
Epoch 9: Train Loss = 54.5098, Val Loss = 50.5295
Epoch 10: Train Loss = 52.1227, Val Loss = 49.0218
Epoch 11: Train Loss = 51.5400, Val Loss = 49.3166
Epoch 12: Train Loss = 51.4064, Val Loss = 49.9826
Epoch 13: Train Loss = 50.9966, Val Loss = 48.5216
Epoch 14: Train Loss = 50.9579, Val Loss = 48.9057
Epoch 15: Train Loss = 50.7145, Val Loss = 47.8009
Epoch 16: Train Loss = 51.1629, Val Loss = 57.0275
Epoch 17: Train Loss = 51.0263, Val Loss = 47.5214
Epoch 18: Train Loss = 50.9372, Val Loss = 49.2975
Epoch 19: Train Loss = 50.7249, Val Loss = 47.3985
Epoch 20: Train Loss = 50.608

In [76]:
def simple_accuracy_check(model, test_loader, device, threshold=0.5):
    """Simple accuracy check focusing on objectness prediction"""
    model.eval()
    correct = 0
    total = 0
    
    with torch.no_grad():
        for batch_images, batch_targets in test_loader:
            batch_images = batch_images.to(device)
            outputs = model(batch_images)
            
            for output, target in zip(outputs, batch_targets):
                # Handle different target formats
                if isinstance(target, torch.Tensor):
                    target = target.to(device)
                    
                    # Extract objectness from prediction
                    pred_objectness = torch.sigmoid(output[0, 0]).item()
                    
                    # Extract objectness from target - handle different possible formats
                    if len(target.shape) == 1 and target.shape[0] >= 1:
                        # If target is [objectness, x_min, y_min, x_max, y_max] format
                        true_objectness = target[0].item()
                    elif len(target.shape) == 2:
                        # If target has extra dimension
                        true_objectness = target[0, 0].item()
                    else:
                        # Fallback - assume first element is objectness
                        true_objectness = target.flatten()[0].item()
                        
                    pred_class = 1 if pred_objectness > threshold else 0
                    true_class = 1 if true_objectness > 0.5 else 0
                    
                    if pred_class == true_class:
                        correct += 1
                    total += 1
                else:
                    print(f"Unexpected target type: {type(target)}")
                    print(f"Target content: {target}")
                    break
    
    if total > 0:
        accuracy = 100 * correct / total
        print(f'Simple Objectness Accuracy: {accuracy:.2f}% ({correct}/{total})')
        return accuracy
    else:
        print("No samples processed. Check your data format.")
        return 0

def load_model(model_path, device):
    """Load a saved model"""
    
    model = ObjectDetectionCNN().to(device)
    
    
    checkpoint = torch.load(model_path, map_location=device)
    model.load_state_dict(checkpoint['model_state_dict'])
    model.eval()
    
    print(f"Model loaded from: {model_path}")
    print(f"Training was stopped at epoch: {checkpoint['epoch']}")
    return model

In [77]:
from datetime import datetime
os.makedirs('saved_models', exist_ok=True)
model_path = f'saved_models/object_detection_model_{datetime.now().strftime("%Y%m%d_%H%M%S")}.pth'
torch.save({
    'model_state_dict': model.state_dict(),
    'optimizer_state_dict': optimizer.state_dict(),
    'epoch': epoch,
    'train_loss': avg_train_loss,
    'val_loss': avg_val_loss,
    'model_architecture': 'ObjectDetectionCNN'
}, model_path)

print(f"Model saved to: {model_path}")

Model saved to: saved_models/object_detection_model_20250528_132238.pth


In [80]:
test_image_dir = "../input/exam-cheating-detection/4000Final/test/images"
test_label_dir = "../input/exam-cheating-detection/4000Final/test/labels"
test_transform = A.Compose([A.Resize(height=320, width=320, p=1.0)])
test_dataset = YOLODataset(
    images_dir=test_image_dir,
    labels_dir=test_label_dir,
    transform = test_transform
)

test_loader = DataLoader(
    test_dataset,
    batch_size=4,
    shuffle=True,
    collate_fn=collate_fn
)
model = load_model('/kaggle/working/saved_models/object_detection_model_20250528_131902.pth',device)
simple_accuracy_check(model,test_loader,device)

Model loaded from: /kaggle/working/saved_models/object_detection_model_20250528_131902.pth
Training was stopped at epoch: 19
Simple Objectness Accuracy: 62.70% (311/496)


62.70161290322581